#### Mixing in Columns in Inheritance Scenarios

In contrast to how `__tablename__` and _other special names_ are handled when used with `declared_attr`, when we _mix in columns and properties_ (e.g. `relationships`, `column properties`, etc.), the __function is invoked for the base class only in the hierarchy__. Below, __only__ the `Person` class will _receive a column called id_; the __mapping will fail__ on `Engineer`, which is _not given a primary key_.

```
@declarative_mixin
class HasId:
    @declared_attr
    def id(cls):
        return Column("id", Integer, primary_key=True)


class Person(HasId, Base):
    __tablename__ = "person"
    discriminator = Column("type", String(50))
    __mapper_args__ = {"polymorphic_on": discriminator}


class Engineer(Person):
    __tablename__ = "engineer"
    primary_language = Column(String(50))
    __mapper_args__ = {"polymorphic_identity": "engineer"}
```

It is usually the case in `joined-table inheritance` that we want __distinctly named columns on each subclass__. However in this case, we may want to have an _id column on every table_, and _have them refer to each other via foreign key_. We can achieve this as a _mixin_ by using the `declared_attr.cascading` _modifier_, which __indicates that the function should be invoked for each class in the hierarchy__, in _almost the same way_ (see warning below) as it does for `__tablename__`.

In [1]:
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import (
    declared_attr, declarative_mixin, declarative_base, has_inherited_table,
)

In [2]:
Base = declarative_base()

In [3]:
@declarative_mixin
class HasIdMixin:
    @declared_attr.cascading
    def id(cls):
        if has_inherited_table(cls):
            return Column(ForeignKey("person.id"), primary_key=True)
        else:
            return Column(Integer, primary_key=True)

In [4]:
class Person(HasIdMixin, Base):
    __tablename__ = "person"
    discriminator = Column("type", String(50))
    __mapper_args__ = {"polymorphic_on": discriminator}

In [5]:
class Engineer(Person):
    __tablename__ = "engineer"
    primary_language = Column(String(50))
    __mapper_args__ = {"polymorphic_identity": "engineer"}

> ##### Warning
> 
> The `declared_attr.cascading` feature __currently does not allow for a subclass to override__ the attribute _with a different function or value_. This is a _current limitation_ in the mechanics of how `@declared_attr` is resolved, and a _warning is emitted_ if this condition is detected. This _limitation_ __does not exist for the special attribute__ names such as `__tablename__`, which __resolve in a different way internally__ than that of `declared_attr.cascading`.